## Social desirability experiment: replicating Daoust findings - data cleaning

Purpose of this notebook: clean up raw data and output analysis datasets

Experiment specifications: <br>
* Daoust experiment:
  + Have you done? Yes=1/No=2/Unsure=3 (Q1.3, Q1.4, Q1.5) <br>
  + Some people XX have you done? Yes=1/Only when neccessary=2/No=3/Unsure=4 (Q2.3, Q2.4, Q2.5) <br><br>
  
* SD benchmark:
  + Individuals are more to blame? Agree=1/disagree=2 (Q3.1) <br>
  + Social conditions are more to blame? Agree=1/disagree=2 (Q3.2) <br>

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
rep_exp = pd.read_csv('../input/rep_exp.csv')

In [3]:
rep_exp.shape

(5234, 34)

In [4]:
rep_exp.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q1.3', 'Q1.4', 'Q1.5', 'Q2.3',
       'Q2.4', 'Q2.5', 'Q3.1', 'Q3.2', 'Q5.1', 'Q5.2', 'Q5.3', 'Q5.4', 'SC0',
       'timeload', 'DeviceIdentifier', 'ResponseID', 'ipaddress'],
      dtype='object')

In [5]:
# columns: select and rename analysis variables
rep_exp = rep_exp[['Finished', 'Q1.3', 'Q1.4', 'Q1.5', 'Q2.3', 'Q2.4', 'Q2.5', 'Q3.1', 'Q3.2', 
                   'Q5.1', 'Q5.2', 'Q5.3', 'Q5.4']]

rep_exp.columns = ['Finished', 'visit_A', 'over_A', 'outdoors_A', 'visit_B', 'over_B', 'outdoors_B', 'blame_individual', 'blame_social', 
                   'sex', 'marital', 'age_group', 'education']

In [6]:
# rows: remove breakoffs
rep_exp = rep_exp.loc[rep_exp['Finished']==1, :]

In [7]:
rep_exp.shape

(4686, 13)

In [8]:
# rows: remove unsure -> save as analysis dataset "df"
df = rep_exp[(rep_exp['visit_A'] != 3) & (rep_exp['over_A'] != 3) & (rep_exp['outdoors_A'] != 3) & 
             (rep_exp['visit_B'] != 4) & (rep_exp['over_B'] != 4) & (rep_exp['outdoors_B'] != 4)]

In [9]:
df.shape

(4633, 13)

In [10]:
# rows: check demographic frequencies - at analysis stage: consider whether 1) remove missing demographics, 2) recode demographics
demographic_cols = ['sex', 'marital', 'age_group', 'education']

for col in demographic_cols:
    print(df[col].value_counts(dropna=False).sort_index())
    print("................")

1.0    2465
2.0    2088
NaN      80
Name: sex, dtype: int64
................
1.0    3010
2.0     384
3.0     657
4.0      44
5.0     465
NaN      73
Name: marital, dtype: int64
................
1.0       5
2.0      86
3.0     191
4.0     325
5.0     674
6.0    1335
7.0    1948
NaN      69
Name: age_group, dtype: int64
................
1.0      53
2.0     759
3.0    1729
4.0    1245
5.0     788
NaN      59
Name: education, dtype: int64
................


In [11]:
# rows: remove those that are age<18
df = df.loc[~np.isin(df['age_group'], [1]), :]

In [12]:
df.shape

(4628, 13)

In [13]:
# reset and add index column "id"
df = df.reset_index(drop=True)
df['id'] = range(1, len(df) + 1)

In [14]:
df.head()

,Finished,visit_A,over_A,outdoors_A,visit_B,over_B,outdoors_B,blame_individual,blame_social,sex,marital,age_group,education,id
0,1,NaN,NaN,NaN,3.0,3.0,1.0,NaN,2.0,1.0,3.0,7.0,5.0,1
1,1,2.0,1.0,2.0,NaN,NaN,NaN,2.0,NaN,1.0,1.0,7.0,5.0,2
2,1,2.0,2.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,1.0,7.0,3.0,3
3,1,2.0,2.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,3.0,7.0,3.0,4
4,1,NaN,NaN,NaN,3.0,3.0,3.0,NaN,2.0,1.0,1.0,7.0,5.0,5


In [15]:
# separate into two analysis datasets: Daoust experiment and SD benchmark experiment, both with demographics
df_dst = df[['id', 'visit_A', 'over_A', 'outdoors_A', 'visit_B', 'over_B', 'outdoors_B', 'sex', 'marital', 'age_group', 'education']]
df_benchmark = df[['id','blame_individual', 'blame_social', 'sex', 'marital', 'age_group', 'education']]

#### Clean up df_dauost experimental conditions

In [16]:
df_dst.head()

,id,visit_A,over_A,outdoors_A,visit_B,over_B,outdoors_B,sex,marital,age_group,education
0,1,NaN,NaN,NaN,3.0,3.0,1.0,1.0,3.0,7.0,5.0
1,2,2.0,1.0,2.0,NaN,NaN,NaN,1.0,1.0,7.0,5.0
2,3,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1.0,7.0,3.0
3,4,2.0,2.0,1.0,NaN,NaN,NaN,1.0,3.0,7.0,3.0
4,5,NaN,NaN,NaN,3.0,3.0,3.0,1.0,1.0,7.0,5.0


In [17]:
grouping = np.array(['A', 'B'])
row_groupings = []

for index, row in df_dst[['visit_A', 'visit_B']].iterrows():
    boolean_list = ~row.isna()
    selected = grouping[boolean_list]
    try:
        value_selected = selected[0]
    except IndexError:
        # when all columns are nan, no grouping is available
        row_groupings.append('NAN')
        continue
    row_groupings.append(value_selected)

In [18]:
df_dst['condition'] = row_groupings

In [19]:
df_dst['condition'].value_counts(dropna=False).sort_index()

A    2309
B    2319
Name: condition, dtype: int64

In [20]:
df_dst.head()

,id,visit_A,over_A,outdoors_A,visit_B,over_B,outdoors_B,sex,marital,age_group,education,condition
0,1,NaN,NaN,NaN,3.0,3.0,1.0,1.0,3.0,7.0,5.0,B
1,2,2.0,1.0,2.0,NaN,NaN,NaN,1.0,1.0,7.0,5.0,A
2,3,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1.0,7.0,3.0,A
3,4,2.0,2.0,1.0,NaN,NaN,NaN,1.0,3.0,7.0,3.0,A
4,5,NaN,NaN,NaN,3.0,3.0,3.0,1.0,1.0,7.0,5.0,B


In [21]:
# recode "No" before mixing up
A_cols = ['visit_A', 'over_A', 'outdoors_A'] # recode 2 into 9
B_cols = ['visit_B', 'over_B', 'outdoors_B'] # recode 3 into 9

for col in A_cols:
    df_dst[col] = df_dst[[col]].replace([2], [9])
    
for col in B_cols:
    df_dst[col] = df_dst[[col]].replace([3], [9])

In [22]:
df_dst['visit'] = df_dst[['visit_A', 'visit_B']].fillna(0).sum(axis=1).astype(int)
df_dst['over'] = df_dst[['over_A', 'over_B']].fillna(0).sum(axis=1).astype(int)
df_dst['outdoors'] = df_dst[['outdoors_A', 'outdoors_B']].fillna(0).sum(axis=1).astype(int)

In [23]:
df_dst = df_dst[['id', 'condition', 'visit', 'over', 'outdoors', 'sex', 'marital', 'age_group', 'education']]

In [24]:
df_dst.head()

,id,condition,visit,over,outdoors,sex,marital,age_group,education
0,1,B,9,9,1,1.0,3.0,7.0,5.0
1,2,A,9,1,9,1.0,1.0,7.0,5.0
2,3,A,9,9,9,1.0,1.0,7.0,3.0
3,4,A,9,9,1,1.0,3.0,7.0,3.0
4,5,B,9,9,9,1.0,1.0,7.0,5.0


In [25]:
df_dst.shape

(4628, 9)

In [26]:
df_dst.to_csv('../output/df_dst.csv', index=False)

#### Clean up df_benchmark experimental conditions

In [27]:
df_benchmark.head()

,id,blame_individual,blame_social,sex,marital,age_group,education
0,1,NaN,2.0,1.0,3.0,7.0,5.0
1,2,2.0,NaN,1.0,1.0,7.0,5.0
2,3,NaN,2.0,1.0,1.0,7.0,3.0
3,4,1.0,NaN,1.0,3.0,7.0,3.0
4,5,NaN,2.0,1.0,1.0,7.0,5.0


In [28]:
pd.crosstab(df_benchmark['blame_individual'], df_benchmark['blame_social'])

""


In [29]:
grouping = np.array(['A', 'B'])
row_groupings = []

for index, row in df_benchmark[['blame_individual', 'blame_social']].iterrows():
    boolean_list = ~row.isna()
    selected = grouping[boolean_list]
    try:
        value_selected = selected[0]
    except IndexError:
        # when all columns are nan, no grouping is available
        row_groupings.append('NAN')
        continue
    row_groupings.append(value_selected)

In [30]:
df_benchmark['condition'] = row_groupings

In [31]:
df_benchmark['condition'].value_counts(dropna=False).sort_index()

A    2336
B    2292
Name: condition, dtype: int64

In [32]:
df_benchmark.head()

,id,blame_individual,blame_social,sex,marital,age_group,education,condition
0,1,NaN,2.0,1.0,3.0,7.0,5.0,B
1,2,2.0,NaN,1.0,1.0,7.0,5.0,A
2,3,NaN,2.0,1.0,1.0,7.0,3.0,B
3,4,1.0,NaN,1.0,3.0,7.0,3.0,A
4,5,NaN,2.0,1.0,1.0,7.0,5.0,B


In [33]:
df_benchmark['blame'] = df_benchmark[['blame_individual', 'blame_social']].fillna(0).sum(axis=1).astype(int)

In [34]:
df_benchmark = df_benchmark[['id', 'condition', 'blame', 'sex', 'marital', 'age_group', 'education']]

In [35]:
df_benchmark.head()

,id,condition,blame,sex,marital,age_group,education
0,1,B,2,1.0,3.0,7.0,5.0
1,2,A,2,1.0,1.0,7.0,5.0
2,3,B,2,1.0,1.0,7.0,3.0
3,4,A,1,1.0,3.0,7.0,3.0
4,5,B,2,1.0,1.0,7.0,5.0


In [36]:
df_benchmark.to_csv('../output/df_benchmark.csv', index=False)